# Task
Write Python code to connect to Google Drive, list files in the "MIMIC dataset" folder, and load the data from the files in that folder.

## Connect to google drive

### Subtask:
Generate code to mount Google Drive to the Colab environment.


**Reasoning**:
Mount Google Drive to the Colab environment using the `google.colab.drive` module.



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Connect to google drive

### Subtask:
Retry generating code to mount Google Drive to the Colab environment, addressing potential credential issues.


**Reasoning**:
Attempt to mount Google Drive again, explicitly forcing a remount to address potential credential issues as instructed.



In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## List files in mimic dataset folder

### Subtask:
Generate code to list the files present in the specified MIMIC dataset folder to verify access and identify data files.


**Reasoning**:
Import the `os` module, define the folder path, list the files and directories in the folder, and print the list.



In [7]:
import os

folder_path = '/content/drive/My Drive/MIMIC Dataset'
try:
    files_and_folders = os.listdir(folder_path)
    print("Files and folders in 'MIMIC dataset':")
    for item in files_and_folders:
        print(item)
except FileNotFoundError:
    print(f"Error: The folder '{folder_path}' was not found. Please check the path.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Files and folders in 'MIMIC dataset':
ADMISSIONS_sorted.csv
D_ICD_DIAGNOSES.csv
DIAGNOSES_ICD_sorted.csv
NOTEEVENTS_sorted.csv
PATIENTS_sorted.csv


In [31]:
import pandas as pd
from datetime import datetime

# Define the folder path
folder_path = '/content/drive/My Drive/MIMIC Dataset'

# Load MIMIC-III tables
notes = pd.read_csv(f"{folder_path}/NOTEEVENTS_sorted.csv", low_memory=False)
diagnoses = pd.read_csv(f"{folder_path}/DIAGNOSES_ICD_sorted.csv", low_memory=False)
diag_dict = pd.read_csv(f"{folder_path}/D_ICD_DIAGNOSES.csv", low_memory=False)
admissions = pd.read_csv(f"{folder_path}/ADMISSIONS_sorted.csv", low_memory=False)
patients = pd.read_csv(f"{folder_path}/PATIENTS_sorted.csv", low_memory=False)

# Optional: Sample small data if testing
# notes = notes.sample(10000)
# diagnoses = diagnoses.sample(10000)

# Filter for only Discharge summaries or Physician notes (summary + diagnostic reasoning)
notes_filtered = notes[notes["CATEGORY"].isin(["Discharge summary", "Physician"])].copy()

# Drop rows with empty text or HADM_ID
notes_filtered = notes_filtered.dropna(subset=["TEXT", "HADM_ID"])

# Merge notes with admissions to get SUBJECT_ID and ADMITTIME
notes_merged = notes_filtered.merge(admissions[["HADM_ID", "SUBJECT_ID", "ADMITTIME"]], on="HADM_ID", how="left")

# Merge with patients to get DOB and gender
notes_merged = notes_merged.merge(patients[["SUBJECT_ID", "DOB", "GENDER"]], left_on='SUBJECT_ID_y', right_on='SUBJECT_ID', how="left")

# Calculate AGE at admission
def calculate_age(row):
    try:
        admit_time = pd.to_datetime(row["ADMITTIME"])
        dob = pd.to_datetime(row["DOB"])
        age = (admit_time - dob).days // 365
        return min(age, 90)  # age capped at 90
    except:
        return None

notes_merged["AGE"] = notes_merged.apply(calculate_age, axis=1)

# Merge with diagnosis codes and long titles
merged_diag = diagnoses.merge(diag_dict, on="ICD9_CODE", how="left")

# Aggregate all diagnosis codes and long titles for each HADM_ID
aggregated_diag = merged_diag.groupby("HADM_ID").agg({
    "ICD9_CODE": lambda x: list(x),
    "LONG_TITLE": lambda x: list(x)
}).reset_index()

# Merge notes with aggregated diagnosis
# This will still have multiple rows per HADM_ID if there are multiple notes,
# but each row for that HADM_ID will have the full list of diagnoses
final_df = notes_merged.merge(aggregated_diag[["HADM_ID", "ICD9_CODE", "LONG_TITLE"]], on="HADM_ID", how="left")

# Final cleanup - renaming columns to reflect combined info if necessary, or keep as list
# Keeping as list for now, column names remain the same as they represent the list
final_df = final_df[["HADM_ID", "SUBJECT_ID_y", "AGE", "GENDER", "TEXT", "ICD9_CODE", "LONG_TITLE"]]
final_df.rename(columns={"TEXT": "NOTES", "LONG_TITLE": "DIAGNOSIS_DESC", "SUBJECT_ID_y": "SUBJECT_ID"}, inplace=True)

# Apply the cleaning function to the 'NOTES' column (assuming clean_notes is defined elsewhere)
# Make sure clean_notes function is available in the environment
try:
    final_df['NOTES'] = final_df['NOTES'].apply(clean_notes)
    print("Notes cleaned using the existing clean_notes function.")
except NameError:
    print("clean_notes function not found. Skipping text cleaning.")


# Save to CSV
# Note: Saving lists to CSV might require specific handling depending on desired format
# For simplicity, saving as is might represent lists as strings like "['code1', 'code2']"
final_df.to_csv("cleaned_mimic_for_clinical_reasoning_with_all_diagnoses.csv", index=False)


print("✅ Done! Extracted dataset saved as 'cleaned_mimic_for_clinical_reasoning_with_all_diagnoses.csv'")
print(f"Total records: {len(final_df)}")
print("\nFirst 5 rows of the final DataFrame with aggregated diagnoses:")
display(final_df.head())

Notes cleaned using the existing clean_notes function.
✅ Done! Extracted dataset saved as 'cleaned_mimic_for_clinical_reasoning_with_all_diagnoses.csv'
Total records: 13063

First 5 rows of the final DataFrame with aggregated diagnoses:


HADM_ID  SUBJECT_ID   AGE GENDER  \
0  145834.0         3.0  76.0      M   
1  185777.0         4.0  47.0      F   
2  107064.0         6.0  65.0      F   
3  150750.0         9.0  41.0      M   
4  150750.0         9.0  41.0      M   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [32]:
print("Columns in notes_merged:")
print(notes_merged.columns)
print("\nColumns in patients:")
print(patients.columns)

Columns in notes_merged:
Index(['ROW_ID', 'SUBJECT_ID_x', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT',
       'SUBJECT_ID_y', 'ADMITTIME', 'SUBJECT_ID', 'DOB', 'GENDER', 'AGE'],
      dtype='object')

Columns in patients:
Index(['ROW_ID', 'SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN',
       'EXPIRE_FLAG'],
      dtype='object')


## Print notes from the cleaned data

### Subtask:
Generate code to display the 'NOTES' column of the `final_df` DataFrame.

**Reasoning**:
Access the 'NOTES' column of the `final_df` DataFrame and display its content.

In [33]:
import pandas as pd

# Set display options to show more characters in the 'NOTES' column
pd.set_option('display.max_colwidth', None)

print("Notes from the cleaned data:")
display(final_df['NOTES'])

Notes from the cleaned data:


0             \n\n     Sex:  M\n\nService:  Medicine\n\nCHIEF COMPLAINT:  Admitted from rehabilitation for\nhypotension (systolic blood pressure to the 70s) and\ndecreased urine output.\n\nHISTORY OF PRESENT ILLNESS:  The patient is a 76-year-old\nmale who had been hospitalized at the  from  through  of \nafter undergoing a left femoral-AT bypass graft and was\nsubsequently discharged to a rehabilitation facility.\n\nOn , he presented again to the  after being found to have a systolic\nblood pressure in the 70s and no urine output for 17 hours.\nA Foley catheter placed at the rehabilitation facility\nyielded 100 cc of murky/brown urine.  There may also have\nbeen purulent discharge at the penile meatus at this time.\n\nOn presentation to the Emergency Department, the patient was\nwithout subjective complaints.  In the Emergency Department,\nhe was found to have systolic blood pressure of 85.  He was\ngiven 6 liters of intravenous fluids and transiently started\non dopamine for a systolic blood pressure in the 80.s\n\nPAST MEDICAL HISTORY:\n 1.  Coronary artery disease with diffuse 3-vessel disease;\nright-dominant, status post proximal left circumflex stent in\n with occlusion of the distal left circumflex;\nstatus post right coronary artery stent on  (no\npercutaneous coronary intervention to 99% diagonal left\ncircumflex, 80% small proximal left anterior descending\nartery, or 80% small distal left anterior descending artery).\n 2.  Congestive heart failure (with an ejection fraction\nof 15% to 20%).\n 3.  Type 2 diabetes with neuropathy.\n 4.  Hypertension.\n 5.  Diverticulosis (found on colonoscopy in ).\n 6.  Alzheimer's dementia.\n 7.  History of gastrointestinal bleed (while the patient was\ntaking eptifibatide).\n 8.  Cardiac risk factors (with a baseline creatinine\nof 1.4 to 1.6).\n 9.  Hypercholesterolemia.\n10.  History of methicillin-resistant Staphylococcus aureus\nand Pseudomonas growth in wound cultures.\n11.  Severe peripheral vascular disease; status post left\nfemoral-AT bypass graft on .\n12.  Chronic nonhealing foot ulcers.\n13.  Recent right pedal cellulitis.\n\nALLERGIES:  The patient has no known drug allergies.\n\nMEDICATIONS ON ADMISSION:\n 1.  Vancomycin 1 g intravenously q.24h. for a level of less\nthan 15 (started on ).\n 2.  Levofloxacin 250 mg p.o. q.d. (started on ).\n 3.  Metronidazole 500 mg p.o. q.8h. (started on ).\n 4.  Heparin 5000 units subcutaneous b.i.d.\n 5.  Simvastatin 40 mg p.o. q.d.\n 6.  Lisinopril 5 mg p.o. q.d.\n 7.  Furosemide 40 mg p.o. q.d.\n 8.  Vitamin E 400 IU p.o. q.d.\n 9.  Atenolol 25 mg p.o. q.d.\n10.  Pantoprazole 40 mg p.o. q.d.\n11.  Ascorbic acid 500 mg p.o. b.i.d.\n12.  NPH 17 units b.i.d.\n13.  Regular insulin sliding-scale.\n14.  Bisacodyl 10 mg p.o./p.r. as needed.\n15.  Docusate 100 mg p.o. b.i.d.\n16.  Percocet 5/325 mg one tablet p.o. q.4-6h. as needed for\npain.\n17.  Aspirin 81 mg p.o. q.d.\n18.  Metoprolol 75 mg p.o. b.i.d.\n\nSOCIAL HISTORY:  The patient is retired and had been living\nat home with his wife prior to his admission to the hospital\non ; he had been living at  for the day prior to admission.  He is a\nsocial drinker and has a 40-pack-year smoking history;\nalthough, he quit smoking 20 years ago.\n\nPHYSICAL EXAMINATION ON PRESENTATION:   Initial physical\nexamination revealed temperature was 96.1 degrees Fahrenheit,\nheart rate was 83, blood pressure was 124/42 (following\nadministration of 3 liters of normal saline), respiratory\nwas 24, and his oxygen saturation was 100% on 2 liters nasal\ncannula.  His heart had a regular rate and rhythm.  There\nwere normal first and second heart sounds.  There was a 2/6\nsystolic ejection murmur, and there were no rubs or gallops.\nHis lungs were clear to auscultation bilaterally.  His\nabdomen was soft, nontender, and nondistended, and there were\nhypoactive bowel sounds.  He had a palpable bypass graft\npulse,  dorsalis pedis and posterior tibialis\npulses bilaterally; and his surgical incision was c

In [34]:
final_df.head()

HADM_ID  SUBJECT_ID   AGE GENDER  \
0  145834.0         3.0  76.0      M   
1  185777.0         4.0  47.0      F   
2  107064.0         6.0  65.0      F   
3  150750.0         9.0  41.0      M   
4  150750.0         9.0  41.0      M   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

## Remove sensitive information from notes

### Subtask:
Generate code to remove admission date, discharge date, DOB, and sex from the 'NOTES' column.

**Reasoning**:
Define a function that uses regular expressions to find and replace date and sex patterns in the text. Apply this function to the 'NOTES' column of the `final_df` DataFrame to remove the sensitive information.

In [35]:
import re

def clean_notes(text):
    # Remove Admission Date
    text = re.sub(r"Admission Date:\s*\[\*\*.*?\*\*\]", "", text)
    # Remove Discharge Date
    text = re.sub(r"Discharge Date:\s*\[\*\*.*?\*\*\]", "", text)
    # Remove DOB (Date of Birth) - assuming a similar pattern to the dates
    text = re.sub(r"DOB:\s*\[\*\*.*?\*\*\]", "", text)
    # Remove "Date of Birth"
    text = re.sub(r"Date of Birth:\s*\[\*\*.*?\*\*\]", "", text)
    # Remove Sex (assuming it might appear as "Sex: [**X**]" or similar, adjust regex if needed)
    text = re.sub(r"Sex:\s*\[\*\*.*?\*\*\]", "", text)
    # Remove Name (as seen in the head of final_df)
    text = re.sub(r"Name:\s*\[\*\*.*?\*\*\](?:,\s*\[\*\*.*?\*\*\])?", "", text)
    # Remove content within [** and **]
    text = re.sub(r"\[\*\*.*?\*\*\]", "", text)

    return text

# Apply the cleaning function to the 'NOTES' column
final_df['NOTES'] = final_df['NOTES'].apply(clean_notes)

print("Sensitive information and bracketed content removed from 'NOTES' column.")
# Display the first few rows of the modified 'NOTES' column to verify
display(final_df['NOTES'].head())

Sensitive information and bracketed content removed from 'NOTES' column.


0         \n\n     Sex:  M\n\nService:  Medicine\n\nCHIEF COMPLAINT:  Admitted from rehabilitation for\nhypotension (systolic blood pressure to the 70s) and\ndecreased urine output.\n\nHISTORY OF PRESENT ILLNESS:  The patient is a 76-year-old\nmale who had been hospitalized at the  from  through  of \nafter undergoing a left femoral-AT bypass graft and was\nsubsequently discharged to a rehabilitation facility.\n\nOn , he presented again to the  after being found to have a systolic\nblood pressure in the 70s and no urine output for 17 hours.\nA Foley catheter placed at the rehabilitation facility\nyielded 100 cc of murky/brown urine.  There may also have\nbeen purulent discharge at the penile meatus at this time.\n\nOn presentation to the Emergency Department, the patient was\nwithout subjective complaints.  In the Emergency Department,\nhe was found to have systolic blood pressure of 85.  He was\ngiven 6 liters of intravenous fluids and transiently started\non dopamine for a systolic blood pressure in the 80.s\n\nPAST MEDICAL HISTORY:\n 1.  Coronary artery disease with diffuse 3-vessel disease;\nright-dominant, status post proximal left circumflex stent in\n with occlusion of the distal left circumflex;\nstatus post right coronary artery stent on  (no\npercutaneous coronary intervention to 99% diagonal left\ncircumflex, 80% small proximal left anterior descending\nartery, or 80% small distal left anterior descending artery).\n 2.  Congestive heart failure (with an ejection fraction\nof 15% to 20%).\n 3.  Type 2 diabetes with neuropathy.\n 4.  Hypertension.\n 5.  Diverticulosis (found on colonoscopy in ).\n 6.  Alzheimer's dementia.\n 7.  History of gastrointestinal bleed (while the patient was\ntaking eptifibatide).\n 8.  Cardiac risk factors (with a baseline creatinine\nof 1.4 to 1.6).\n 9.  Hypercholesterolemia.\n10.  History of methicillin-resistant Staphylococcus aureus\nand Pseudomonas growth in wound cultures.\n11.  Severe peripheral vascular disease; status post left\nfemoral-AT bypass graft on .\n12.  Chronic nonhealing foot ulcers.\n13.  Recent right pedal cellulitis.\n\nALLERGIES:  The patient has no known drug allergies.\n\nMEDICATIONS ON ADMISSION:\n 1.  Vancomycin 1 g intravenously q.24h. for a level of less\nthan 15 (started on ).\n 2.  Levofloxacin 250 mg p.o. q.d. (started on ).\n 3.  Metronidazole 500 mg p.o. q.8h. (started on ).\n 4.  Heparin 5000 units subcutaneous b.i.d.\n 5.  Simvastatin 40 mg p.o. q.d.\n 6.  Lisinopril 5 mg p.o. q.d.\n 7.  Furosemide 40 mg p.o. q.d.\n 8.  Vitamin E 400 IU p.o. q.d.\n 9.  Atenolol 25 mg p.o. q.d.\n10.  Pantoprazole 40 mg p.o. q.d.\n11.  Ascorbic acid 500 mg p.o. b.i.d.\n12.  NPH 17 units b.i.d.\n13.  Regular insulin sliding-scale.\n14.  Bisacodyl 10 mg p.o./p.r. as needed.\n15.  Docusate 100 mg p.o. b.i.d.\n16.  Percocet 5/325 mg one tablet p.o. q.4-6h. as needed for\npain.\n17.  Aspirin 81 mg p.o. q.d.\n18.  Metoprolol 75 mg p.o. b.i.d.\n\nSOCIAL HISTORY:  The patient is retired and had been living\nat home with his wife prior to his admission to the hospital\non ; he had been living at  for the day prior to admission.  He is a\nsocial drinker and has a 40-pack-year smoking history;\nalthough, he quit smoking 20 years ago.\n\nPHYSICAL EXAMINATION ON PRESENTATION:   Initial physical\nexamination revealed temperature was 96.1 degrees Fahrenheit,\nheart rate was 83, blood pressure was 124/42 (following\nadministration of 3 liters of normal saline), respiratory\nwas 24, and his oxygen saturation was 100% on 2 liters nasal\ncannula.  His heart had a regular rate and rhythm.  There\nwere normal first and second heart sounds.  There was a 2/6\nsystolic ejection murmur, and there were no rubs or gallops.\nHis lungs were clear to auscultation bilaterally.  His\nabdomen was soft, nontender, and nondistended, and there were\nhypoactive bowel sounds.  He had a palpable bypass graft\npulse,  dorsalis pedis and posterior tibialis\npulses bilaterally; and his surgical incision was clean

In [36]:
# Create a new DataFrame by dropping 'HADM_ID' and 'SUBJECT_ID' columns
df_without_ids = final_df.drop(columns=['HADM_ID', 'SUBJECT_ID'])

# Drop rows where 'DIAGNOSIS_DESC' or 'NOTES' are null
df_cleaned = df_without_ids.dropna(subset=['DIAGNOSIS_DESC', 'NOTES']).copy()

print(f"Original number of rows: {len(df_without_ids)}")
print(f"Number of rows after dropping nulls: {len(df_cleaned)}")

print("\nFirst 5 rows of the cleaned DataFrame:")
display(df_cleaned.head())

# Save the new DataFrame to a new CSV file
output_csv_path = "cleaned_mimic_notes_only.csv"
df_cleaned.to_csv(output_csv_path, index=False)

print(f"✅ Done! New dataset without HADM_ID and SUBJECT_ID saved as '{output_csv_path}'")
print("First 5 rows of the new DataFrame:")
display(df_cleaned.head())

Original number of rows: 13063
Number of rows after dropping nulls: 13036

First 5 rows of the cleaned DataFrame:


AGE GENDER  \
0  76.0      M   
1  47.0      F   
2  65.0      F   
3  41.0      M   
4  41.0      M   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

✅ Done! New dataset without HADM_ID and SUBJECT_ID saved as 'cleaned_mimic_notes_only.csv'
First 5 rows of the new DataFrame:


AGE GENDER  \
0  76.0      M   
1  47.0      F   
2  65.0      F   
3  41.0      M   
4  41.0      M   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [37]:
print("Diagnosis Descriptions:")
display(df_cleaned['DIAGNOSIS_DESC'])

Diagnosis Descriptions:


,DIAGNOSIS_DESC
0,"[Unspecified protein-calorie malnutrition, Cellulitis and abscess of leg, except foot, Congestive heart failure, unspecified, Subendocardial infarction, initial episode of care, Other primary cardiomyopathies, Acute kidney failure, unspecified, Other shock without mention of trauma, Unspecified septicemia, Cardiac arrest]"
1,"[Antiviral drugs causing adverse effects in therapeutic use, Human immunodeficiency virus [HIV] disease, Pneumocystosis, Cachexia, Alkalosis, Bacteremia, Cirrhosis of liver without mention of alcohol, Methicillin susceptible Staphylococcus aureus in conditions classified elsewhere and of unspecified site, Infection with microorganisms resistant to penicillins]"
2,"[Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease, nan, Peripheral vascular complications, not elsewhere classified, nan, Hyperpotassemia, Anemia, unspecified, Disorders of phosphorus metabolism, Personal history of tobacco use]"
3,"[Intracerebral hemorrhage, Pneumonitis due to inhalation of food or vomitus, Congestive heart failure, unspecified, Acute kidney failure, unspecified, nan, Unspecified essential hypertension]"
4,"[Intracerebral hemorrhage, Pneumonitis due to inhalation of food or vomitus, Congestive heart failure, unspecified, Acute kidney failure, unspecified, nan, Unspecified essential hypertension]"
...,...
13031,"[Other chronic sinusitis, Unspecified essential hypertension, Alcohol withdrawal, Acute kidney failure, unspecified, Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled, Congestive heart failure, unspecified, Methicillin susceptible pneumonia due to Staphylococcus aureus, Acute respiratory failure, Subendocardial infarction, initial episode of care]"
13032,"[Other chronic sinusitis, Unspecified essential hypertension, Alcohol withdrawal, Acute kidney failure, unspecified, Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled, Congestive heart failure, unspecified, Methicillin susceptible pneumonia due to Staphylococcus aureus, Acute respiratory failure, Subendocardial infarction, initial episode of care]"
13033,"[Other abnormal glucose, Family history of ischemic heart disease, Anemia, unspecified, Unspecified essential hypertension, Pure hypercholesterolemia, Percutaneous transluminal coronary angioplasty status, Intermediate coronary syndrome, Coronary atherosclerosis of native coronary artery, Old myocardial infarction]"
13034,"[Unspecified sleep apnea, Morbid obesity, Unspecified essential hypertension, Congestive heart failure, unspecified, Chronic airway obstruction, not elsewhere classified, Acute respiratory failure, Pneumonitis due to inhalation of food or vomitus, Diabetes with other specified manifestations, type II or unspecified type, uncontrolled]"


## Prepare data for LLM fine-tuning

### Subtask:
Generate code to convert the `final_df` DataFrame into a JSON format suitable for LLM fine-tuning, using 'NOTES' as instruction and 'DIAGNOSIS_DESC' as output.

**Reasoning**:
Define a function to format each row into the desired JSON structure. Apply this function to the DataFrame to create a list of dictionaries. Import the `json` library and save the list as a JSON file.

In [40]:
import json
import pandas as pd # Import pandas if not already imported

def format_for_llm(row):
    """Formats a DataFrame row into the desired JSON structure for LLM training."""
    # Ensure 'NOTES' is treated as string and handle potential NaNs
    notes_text = str(row['NOTES']) if pd.notna(row['NOTES']) else ""

    # Handle the list in 'DIAGNOSIS_DESC' - convert list to string representation
    diagnosis_list = row['DIAGNOSIS_DESC']
    if isinstance(diagnosis_list, list):
        # Join the list elements into a string, e.g., separated by commas or a newline
        diagnosis_text = ", ".join(map(str, diagnosis_list))
    elif pd.notna(diagnosis_list):
        # Handle cases where it might not be a list but not NaN
        diagnosis_text = str(diagnosis_list)
    else:
        diagnosis_text = ""


    return {
        "instruction": notes_text,
        "input": "",  # Optional input field
        "output": diagnosis_text
    }

# Apply the function to each row and convert the result to a list
llm_data = final_df.apply(format_for_llm, axis=1).tolist()

# Define the output JSON file path
output_json_path = "mimic_llm_training_data.json"

# Save the data to a JSON file
with open(output_json_path, 'w') as f:
    json.dump(llm_data, f, indent=2)

print(f"✅ Done! Data formatted and saved to '{output_json_path}'")

# Display the first 5 entries of the generated JSON data for verification
print("\nFirst 5 entries of the generated JSON data:")
print(json.dumps(llm_data[:5], indent=2))

✅ Done! Data formatted and saved to 'mimic_llm_training_data.json'

First 5 entries of the generated JSON data:
[
  {
    "instruction": "     \n\n     Sex:  M\n\nService:  Medicine\n\nCHIEF COMPLAINT:  Admitted from rehabilitation for\nhypotension (systolic blood pressure to the 70s) and\ndecreased urine output.\n\nHISTORY OF PRESENT ILLNESS:  The patient is a 76-year-old\nmale who had been hospitalized at the  from  through  of \nafter undergoing a left femoral-AT bypass graft and was\nsubsequently discharged to a rehabilitation facility.\n\nOn , he presented again to the  after being found to have a systolic\nblood pressure in the 70s and no urine output for 17 hours.\nA Foley catheter placed at the rehabilitation facility\nyielded 100 cc of murky/brown urine.  There may also have\nbeen purulent discharge at the penile meatus at this time.\n\nOn presentation to the Emergency Department, the patient was\nwithout subjective complaints.  In the Emergency Department,\nhe was found to ha